<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/gera_patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pacotes
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy import stats
import pickle
from PIL import Image, ImageDraw, ImageFont

In [2]:
from google.colab import drive
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [3]:
# Função para gerar patches sem overlap - tamanho fixo de (256,256)
def split_image(dados_in, input_mes_ano, output_path):
    img = Image.fromarray(np.uint8(dados_in))
    width, height = img.size
    patch_size = 256
    n_width = dados_in.shape[0] // patch_size
    n_height = dados_in.shape[1] // patch_size
    count = 1
    for i in range(n_width):
      box = (i * patch_size, 0, (i + 1) * patch_size, height)
      if i == n_width - 1:
        # Last split includes remaining height
        box = (i * patch_size, 0, width, height)
      for j in range(n_height):
        # Adjust box for each row of splits
        row_box = (box[0], j * patch_size, box[2], (j + 1) * patch_size)
        if j == n_height - 1:
          # Last row includes remaining height
          row_box = (box[0], j * patch_size, box[2], height)
        split = img.crop(row_box)
        # Voltando para array
        salvar = np.asarray(split)
        salvar = salvar.astype('float32')
        # Salvando
        with open(output_path + input_mes_ano + "_patch_" + str(count) + ".pkl", 'wb') as handle:
          pickle.dump(salvar, handle, protocol=pickle.HIGHEST_PROTOCOL)
        if (count % 25) == 0:
          print("count = " + str(count))
        count += 1

In [4]:
path = "/content/drive/My Drive/projeto_DL/dataset_tratado/mes/"
arqs = []
mes_ano = []
for i in ['2017','2018','2019','2020','2021']:
  for j in range(12):
    arqs.append(path+i+"_"+str(j+1)+".pkl")
    mes_ano.append(i+"_"+str(j+1))

print(arqs)
print(mes_ano)

['/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_1.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_2.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_3.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_4.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_5.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_6.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_7.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_8.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_9.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_10.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_11.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2017_12.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2018_1.pkl', '/content/drive/My Drive/projeto_DL/dataset_tratado/mes/2018_2.pkl', '/content/drive/My Drive/proje

In [5]:
for i in range(len(arqs)):
  with open(arqs[i], 'rb') as handle:
    dados = pickle.load(handle)
  split_image(dados, mes_ano[i], '/content/drive/My Drive/projeto_DL/dataset_tratado/patches/')
  print(mes_ano[i])


count = 25
count = 50
count = 75
count = 100
2017_1
count = 25
count = 50
count = 75
count = 100
2017_2
count = 25
count = 50
count = 75
count = 100
2017_3
count = 25
count = 50
count = 75
count = 100
2017_4
count = 25
count = 50
count = 75
count = 100
2017_5
count = 25
count = 50
count = 75
count = 100
2017_6
count = 25
count = 50
count = 75
count = 100
2017_7
count = 25
count = 50
count = 75
count = 100
2017_8
count = 25
count = 50
count = 75
count = 100
2017_9
count = 25
count = 50
count = 75
count = 100
2017_10
count = 25
count = 50
count = 75
count = 100
2017_11
count = 25
count = 50
count = 75
count = 100
2017_12
count = 25
count = 50
count = 75
count = 100
2018_1
count = 25
count = 50
count = 75
count = 100
2018_2
count = 25
count = 50
count = 75
count = 100
2018_3
count = 25
count = 50
count = 75
count = 100
2018_4
count = 25
count = 50
count = 75
count = 100
2018_5
count = 25
count = 50
count = 75
count = 100
2018_6
count = 25
count = 50
count = 75
count = 100
2018_7
count = 2